In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso

from sklearn import datasets
from sklearn.datasets import load_diabetes

import warnings
import pandas as pd

In [2]:
data = pd.read_csv('movies_metadata.csv')
#data = data.fillna('')
data['release_date']

/Users/zzkzzk/Downloads/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
5        1995-12-15
6        1995-12-15
7        1995-12-22
8        1995-12-22
9        1995-11-16
10       1995-11-17
11       1995-12-22
12       1995-12-22
13       1995-12-22
14       1995-12-22
15       1995-11-22
16       1995-12-13
17       1995-12-09
18       1995-11-10
19       1995-11-21
20       1995-10-20
21       1995-10-27
22       1995-10-06
23       1995-10-27
24       1995-10-27
25       1995-12-15
26       1995-10-20
27       1995-09-27
28       1995-05-16
29       1995-04-30
            ...    
45436    2010-01-01
45437    2017-06-01
45438    2015-04-02
45439    1944-06-01
45440    2007-01-08
45441    2002-03-07
45442    1905-01-01
45443    1901-01-01
45444    1905-01-01
45445    1906-01-01
45446    1909-01-01
45447    1904-03-05
45448    1904-01-01
45449    2005-09-13
45450    1900-01-01
45451    1900-01-01
45452    1981-01-01
45453    2017-07-07
45454    2015-01-11


In [47]:
#print (data['release_date'][6000][0:4])
#data['release_date'][45465][0:4] == "2017"
data['release_year'] = data['release_date'].apply(lambda x : str(x).split('-')[0])
data_2016 = data[data['release_year'] == "2016"]
#latest_data = data[(data['release_date'][i][0:4] == "2017")] for i in range(0,45466)
data_2016 = data_2016[data_2016['revenue'] != 0.000000e+00]
data_2016 = data_2016[data_2016['original_language'] == "en"]
all_titles = data_2016['original_title']

In [48]:
import requests

def retrieve_html(url):
    """
    Return the raw HTML at the specified URL.

    Args:
        url (string): 

    Returns:
        status_code (integer):
        raw_html (string): the raw HTML content of the response, properly encoded according to the HTTP headers.
    """
    raw_html = requests.get(url).text
    status_code = requests.get(url).status_code
    return (status_code, raw_html)

    

In [7]:
url = 'https://www.rottentomatoes.com/m/searching_2018/reviews/?sort='
(status,searching_html) = retrieve_html(url) 
print(searching_html)

<!DOCTYPE html>
<html lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/" >
    <head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#"> <script src="//cdn.optimizely.com/js/594670329.js"></script> <link href="https://d2a5cgar23scu2.cloudfront.net/v/less/?f=/styles/rt_redesign.less" type="text/css" rel="stylesheet"/><meta http-equiv="Content-Type" content="text/html; charset=utf-8"/> <meta name="viewport" content="width=device-width,initial-scale=1"> <meta name="google-site-verification" content="VPPXtECgUUeuATBacnqnCm4ydGO99reF-xgNklSbNbc"/> <meta name="msvalidate.01" content="034F16304017CA7DCF45D43850915323"/> <link href="//d2a5cgar23scu2.cloudfront.net/static/images/icons/apple-touch-icon.png" rel="apple-touch-icon"/> <link href="//d2a5cgar23scu2.cloudfront.net/static/images/icons/favicon.ico" rel="shortcut icon" type="image/x-icon"/> <link href="//d2a5cgar23scu2.cloudfront.net/static/styles/b

In [10]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(searching_html,"lxml")
all_para = soup.findAll("div", {"class": "the_review"})
all_para

[<div class="the_review"> At a certain point, Searching runs out of ways to organically use its medium and the tension erodes. The first two acts, however, are engrossing and remind the viewer that John Cho should be a star. </div>,
 <div class="the_review"> For cases like Searching, innovation isn't enough.</div>,
 <div class="the_review"> With clever and immersive storytelling, great performances, rich characterizations, surprising twists and ample amounts of food for thought, director Aneesh Chaganty has made a great feature debut with Searching. </div>,
 <div class="the_review"> Mind-boggling, electric, and hugely-entertaining, this is one of the best surprises this year. </div>,
 <div class="the_review"> An out-of-left-field crowd-pleaser that just might be the best "cyber-thriller" ever made.</div>,
 <div class="the_review"> The screen-centric format allows the viewer to play detective right along with Cho, the reams of emails and text messages making for a strangely interactive 

In [40]:
def extract_reviews(movie_name):
    url = 'https://www.rottentomatoes.com/m/' + str(movie_name) + '/reviews/'
    raw_html = requests.get(url).text
    status_code = requests.get(url).status_code
    #print ('status code',  status_code)
    soup = BeautifulSoup(raw_html,"lxml")
    all_para = soup.findAll("div", {"class": "the_review"})
    #print (type(soup)
    print ()
    return all_para, status_code

In [99]:
def extract2016_reviews(movie_name):
    url = 'https://www.rottentomatoes.com/m/' + str(movie_name) + '_2016' + '/reviews/'
    raw_html = requests.get(url).text
    status_code = requests.get(url).status_code
    #print ('status code',  status_code)
    soup = BeautifulSoup(raw_html,"lxml")
    all_para = soup.findAll("div", {"class": "the_review"})
    #print (type(soup)
    print ()
    return all_para, status_code

In [41]:
extract_reviews('Colossal')[1]

200

In [102]:
#print (all_titles)

In [84]:
import re
all_titles_list = all_titles.tolist()
updated_titles = []
for i in all_titles_list:
    #print (i)
    updated_titles.append( '_'.join(list(filter(lambda x: len(x) != 0, re.split('[^0-9a-zA-Z]', i)))))
updated_titles
#list(filter(lambda x: len(x) != 0, re.split('[^0-9a-zA-Z]', "Ben-Hur")))

['Ben_Hur',
 'Warcraft',
 'Deadpool',
 'Captain_America_Civil_War',
 'Doctor_Strange',
 'X_Men_Apocalypse',
 'The_Bronze',
 'Midnight_Special',
 'Allegiant',
 'Fantastic_Beasts_and_Where_to_Find_Them',
 'The_Secret_Life_of_Pets',
 'Criminal',
 'Suicide_Squad',
 'Independence_Day_Resurgence',
 'Star_Trek_Beyond',
 'Batman_v_Superman_Dawn_of_Justice',
 'The_Jungle_Book',
 '13_Hours_The_Secret_Soldiers_of_Benghazi',
 'Colonia',
 'The_Witch',
 '3_Generations',
 'Green_Room',
 'Man_Down',
 'Demolition',
 'Silence',
 'Hail_Caesar',
 'The_5th_Wave',
 'Zoolander_2',
 'How_to_Be_Single',
 'Fifty_Shades_of_Black',
 'Nocturnal_Animals',
 'Kung_Fu_Panda_3',
 'Krisha',
 'Pride_and_Prejudice_and_Zombies',
 'Jane_Got_a_Gun',
 'The_Forest',
 'Norm_of_the_North',
 'Ride_Along_2',
 'Maggie_s_Plan',
 'Eddie_the_Eagle',
 'Exposed',
 'The_Boy',
 'Risen',
 'Holy_Hell',
 'The_Eagle_Huntress',
 'The_Other_Side_of_the_Door',
 'Dirty_Grandpa',
 'The_Finest_Hours',
 'The_Birth_of_a_Nation',
 'Me_Before_You',
 'M

In [100]:
def count_success(titles):
    count = 0
    for movie in titles:
        if extract2016_reviews(movie)[1] == 200:
            count += 1
            print (movie)
    return count

In [103]:
#count_success(updated_titles)

In [106]:
import re

def format_movie_name(movie_name): 
    movie_name = movie_name.replace("&", " and ")
    movie_name = movie_name.replace("'", "")
    word_list = list(filter(lambda x: len(x) != 0, re.split('[^0-9a-zA-Z]', movie_name)))
    s = "_"
    return s.join(word_list)

In [107]:
def get_reviews(selected_df, year):
    url_part_one = "https://www.rottentomatoes.com/m/"
    url_part_two = "/reviews/"
    for movie in selected_df.original_title:
        movie_string = format_movie_name(movie)
        url = url_part_one + movie_string + "_" + str(year) + url_part_two
        response = requests.get(url, allow_redirects=False)
        if response.status_code == 200:
            print ("MULTIPLE: " + movie)
        if response.status_code != 200:
            url = url_part_one + movie_string + url_part_two
            response = requests.get(url)
        if response.status_code == 404:
            print ("cannot find " + movie)

In [110]:
get_reviews(data_2016, 2016)

MULTIPLE: Ben-Hur
MULTIPLE: Doctor Strange
MULTIPLE: Criminal
MULTIPLE: Suicide Squad
MULTIPLE: Batman v Superman: Dawn of Justice
MULTIPLE: The Jungle Book
MULTIPLE: The Witch
MULTIPLE: Green Room
MULTIPLE: Demolition
MULTIPLE: Hail, Caesar!
MULTIPLE: The 5th Wave
MULTIPLE: How to Be Single
MULTIPLE: Krisha
MULTIPLE: The Forest
MULTIPLE: Maggie's Plan
MULTIPLE: Exposed
MULTIPLE: Risen
MULTIPLE: Holy Hell
MULTIPLE: The Birth of a Nation
MULTIPLE: Miles Ahead
MULTIPLE: Race
cannot find 美人鱼
MULTIPLE: Elvis & Nixon
MULTIPLE: Mr. Right
MULTIPLE: Florence Foster Jenkins
MULTIPLE: Keanu
MULTIPLE: Nine Lives
MULTIPLE: Barbershop: The Next Cut
MULTIPLE: Our Kind of Traitor
MULTIPLE: The Boss
MULTIPLE: Precious Cargo
MULTIPLE: Mother's Day
MULTIPLE: The BFG
MULTIPLE: The Meddler
MULTIPLE: The Darkness
cannot find Pelé: Birth of a Legend
MULTIPLE: Ghostbusters
MULTIPLE: Cell
MULTIPLE: Genius
MULTIPLE: Lights Out
MULTIPLE: Pete's Dragon
MULTIPLE: The Duel
MULTIPLE: Goat
MULTIPLE: Satanic
MULTIPLE